In [1]:
# #langchain library
# Install langchain (version compatible with other libraries)
!pip install langchain==0.1.0

# Install LLM-related packages
!pip install accelerate==0.25.0
!pip install transformers==4.36.2
!pip install bitsandbytes

# Install Text Embedding libraries
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding==1.0.1

# Install vectorstore-related packages
!pip install pymupdf==1.23.8

# If you need GPU support with FAISS, try this:
# !pip install faiss-gpu-cu12 # CUDA 12.x, Python 3.8+
# If you're having trouble with `faiss-gpu`, use the CPU version:
# pip install faiss-cpu==1.7.2

# !pip install faiss-cpu==1.7.4
# conda install -c pytorch faiss-gpu

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain-0.1.0-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.1.0-py3-none-any.whl (797 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.2
    Uninstalling langchain-0.1.2:
      Successfully uninstalled langchain-0.1.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import torch 
from langchain import PromptTemplate
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory


## Prompt

In [4]:
from langchain.prompts import PromptTemplate

# Define a template to answer questions about solar energy based on a single article
solar_energy_prompt_template = """
You are a friendly assistant knowledgeable in solar energy. Below is a summary of key details from the article:

**Article Title**: Photovoltaic Solar Energy: Principles and Applications
**Source**: National Renewable Energy Laboratory (NREL)
**Key Concepts**:
- Photovoltaic (PV) solar energy involves converting sunlight directly into electricity using solar panels.
- PV systems are widely used for residential and commercial electricity generation.
- The efficiency of PV cells has improved significantly, making them more affordable and efficient.
- Solar energy is a renewable and clean energy source, reducing reliance on fossil fuels.

Please answer the following question using this information, in a professional, gentle, and informative manner.

Question: {question}
Answer:
""".strip()

PROMPT = PromptTemplate.from_template(
    template = solar_energy_prompt_template
)

PROMPT


PromptTemplate(input_variables=['question'], template='You are a friendly assistant knowledgeable in solar energy. Below is a summary of key details from the article:\n\n**Article Title**: Photovoltaic Solar Energy: Principles and Applications\n**Source**: National Renewable Energy Laboratory (NREL)\n**Key Concepts**:\n- Photovoltaic (PV) solar energy involves converting sunlight directly into electricity using solar panels.\n- PV systems are widely used for residential and commercial electricity generation.\n- The efficiency of PV cells has improved significantly, making them more affordable and efficient.\n- Solar energy is a renewable and clean energy source, reducing reliance on fossil fuels.\n\nPlease answer the following question using this information, in a professional, gentle, and informative manner.\n\nQuestion: {question}\nAnswer:')

In [5]:
PROMPT.format(
    context = "Solar energy is the radiant energy from the Sun's light and heat, which can be harnessed using a range of technologies such as solar electricity, solar thermal energy and solar architecture" ,
    question = "What is solar energy"
)

'You are a friendly assistant knowledgeable in solar energy. Below is a summary of key details from the article:\n\n**Article Title**: Photovoltaic Solar Energy: Principles and Applications\n**Source**: National Renewable Energy Laboratory (NREL)\n**Key Concepts**:\n- Photovoltaic (PV) solar energy involves converting sunlight directly into electricity using solar panels.\n- PV systems are widely used for residential and commercial electricity generation.\n- The efficiency of PV cells has improved significantly, making them more affordable and efficient.\n- Solar energy is a renewable and clean energy source, reducing reliance on fossil fuels.\n\nPlease answer the following question using this information, in a professional, gentle, and informative manner.\n\nQuestion: What is solar energy\nAnswer:'

In [6]:
# generated_prompt.format(
#     context = "Majorly i was inovolved with finance industry, as data analayst",
#     question = "What role and industry were you inolved with?"
# 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader

def process_multiple_pdfs(pdf_files):
    # Initialize the RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700,
        chunk_overlap=100
    )

    # Create a list to store all the chunks
    all_chunks = []

    # Iterate over each PDF file
    for pdf_file in pdf_files:
        # Load the document using PyMuPDFLoader
        loader = PyMuPDFLoader(pdf_file)
        documents = loader.load()

        # Process each document and split into chunks
        for document in documents:
            chunks = text_splitter.split_documents([document])
            # Append all chunks to the list
            all_chunks.extend(chunks)

    # Return the chunks
    return all_chunks

# List of PDFs to process
pdf_files = ["About_solar.pdf", "About_solar2.pdf","type_solar.pdf", "international_conferene.pdf"]

# Process and return all chunks
documents = process_multiple_pdfs(pdf_files)

# Print results
print(f"Total number of chunks: {len(documents)}")
for chunk in documents:
    print(chunk)


Total number of chunks: 4
page_content="Solar energy is the radiant energy from the Sun's light and heat, which can be harnessed using a range of technologies such as solar electricity, \nsolar thermal energy (including solar water heating) and solar architecture. It is an essential source of renewable energy, and its technologies \nare broadly characterized as either passive solar or active solar depending on how they capture and distribute solar energy or convert it into \nsolar power." metadata={'source': 'About_solar.pdf', 'file_path': 'About_solar.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20250314025650Z', 'modDate': '', 'trapped': ''}
page_content='solar energy, radiation from the Sun capable of producing heat, causing chemical\nreactions, or generating electricity. The total amount of solar energy incident\non Earth is vastly in excess of the wo

In [24]:
#possible questions to ask
#what is solar energy
#what are the active solar techniques
#what are passive solar techniques


## Retrieval
* Document loaders : Load documents from many different sources (HTML, PDF, code).
* Document transformers : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
* Text embedding models : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
* Vector stores: there has emerged a need for databases to support efficient storage and searching of these embeddings.
* Retrievers : Once the data is in the database, you still need to retrieve it.
## Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.



In [6]:
# nlp_docs = 'About_solar.pdf'

# loader = PyMuPDFLoader(nlp_docs)
# documents = loader.load()

In [3]:
len(documents)

4

In [22]:
documents[1]

Document(page_content='solar energy, radiation from the Sun capable of producing heat, causing chemical\nreactions, or generating electricity. The total amount of solar energy incident\non Earth is vastly in excess of the world’s current and anticipated energy re-\nquirements. If suitably harnessed, this highly diffused source has the potential\nto satisfy all future energy needs. In the 21st century solar energy has become\nincreasingly attractive as a renewable energy source because of its inexhaustible\nsupply and its nonpolluting character, in stark contrast to the finite fossil fuels\ncoal, petroleum, and natural gas. See also solar power.\n1', metadata={'source': 'About_solar2.pdf', 'file_path': 'About_solar2.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX via pandoc', 'producer': 'xdvipdfmx (20211117)', 'creationDate': "D:20250314112713-00'00'", 'modDate': '', 'trapped': ''})

## Document Transformers

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

for chunk in doc:
    print(chunk)
    print("-" * 20)

page_content="Solar energy is the radiant energy from the Sun's light and heat, which can be harnessed using a range of technologies such as solar electricity, \nsolar thermal energy (including solar water heating) and solar architecture. It is an essential source of renewable energy, and its technologies \nare broadly characterized as either passive solar or active solar depending on how they capture and distribute solar energy or convert it into \nsolar power." metadata={'source': 'About_solar.pdf', 'file_path': 'About_solar.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20250314025650Z', 'modDate': '', 'trapped': ''}
--------------------
page_content='solar energy, radiation from the Sun capable of producing heat, causing chemical\nreactions, or generating electricity. The total amount of solar energy incident\non Earth is vastly in excess of the world’s

In [31]:
len(doc)

3

## Text Embedding Models

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

Note Instructor Model : Huggingface | Paper

In [3]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Set the device (e.g., 'cuda' or 'cpu')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs={"device": device}  # Avoid passing any unsupported arguments like 'token'
)


load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
# !pip install langchain==0.1.2
# !pip install -U sentence-transformers==2.2.2
# !pip install huggingface-hub==0.25.2

## Vector-store

In [12]:
import os
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [24]:
from langchain.vectorstores import FAISS
import os

# Assuming 'documents' is your list of Document objects
texts = [doc.page_content for doc in documents]
metadatas = [doc.metadata for doc in documents]

# Generate embeddings for the texts
embeddings = embedding_model.embed_documents(texts)

# Create a list of tuples (text, embedding)
text_embedding_pairs = zip(texts, embeddings)

# Initialize FAISS Vector Store from embeddings
vector_path = './vector_db_path'  # Replace with your desired path
db_file_name = 'nlp_stanford'
vector_path = os.path.abspath(vector_path)
full_path = os.path.join(vector_path, db_file_name)
print(f"Full path where index is saved: {full_path}")

vectordb = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,
    embedding=embedding_model,
    metadatas=metadatas  # Include metadata separately
)

# Save the FAISS index to a specified local path
vectordb.save_local(
    folder_path=full_path,
    index_name='nlp'  # Default index name
)

print(f"FAISS index saved to: {os.path.join(vector_path, db_file_name)}")


Full path where index is saved: /home/jupyter-st125490/vector_db_path/nlp_stanford
FAISS index saved to: /home/jupyter-st125490/vector_db_path/nlp_stanford


## Retriver

In [25]:
from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = full_path,
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   


In [26]:
query = "What is solar energy?"

# Embed the query using the same embedding model
query_embedding = embedding_model.embed_query(query)

In [28]:
results = vectordb.similarity_search(query, k=2)  # 'k' is the number of similar documents you want

# Print the results
for result in results:
    print(result) 

page_content="Solar energy is the radiant energy from the Sun's light and heat, which can be harnessed using a range of technologies such as solar electricity, \nsolar thermal energy (including solar water heating) and solar architecture. It is an essential source of renewable energy, and its technologies \nare broadly characterized as either passive solar or active solar depending on how they capture and distribute solar energy or convert it into \nsolar power." metadata={'source': 'About_solar.pdf', 'file_path': 'About_solar.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20250314025650Z', 'modDate': '', 'trapped': ''}
page_content='solar energy, radiation from the Sun capable of producing heat, causing chemical\nreactions, or generating electricity. The total amount of solar energy incident\non Earth is vastly in excess of the world’s current and anticipa

In [29]:
query = "When was the international conference done?"

# Embed the query using the same embedding model
query_embedding = embedding_model.embed_query(query)

In [31]:
results = vectordb.similarity_search(query, k=1)  # 'k' is the number of similar documents you want

# Print the results
for result in results:
    print(result) 

page_content='In 2011, the International Energy Agency said that "the development of affordable, inexhaustible and clean solar energy technologies will have \nhuge longer-term benefits. It will increase countries\' energy security through reliance on an indigenous, inexhaustible, and mostly import-\nindependent resource, enhance sustainability, reduce pollution, lower the costs of mitigating global warming. these advantages are global".' metadata={'source': 'international_conferene.pdf', 'file_path': 'international_conferene.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20250314025902Z', 'modDate': '', 'trapped': ''}


In [33]:

vectordb = FAISS.load_local(
    folder_path = full_path,
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   
#ready to use
retriever = vectordb.as_retriever()
retriever.get_relevant_documents("what are two types of solar techniques?")

[Document(page_content='There are two type Active solar and passive solar. Active solar techniques include the use of photovoltaic systems, concentrated solar power, \nand solar water heating to harness the energy. Passive solar techniques include designing a building for better daylighting, selecting materials \nwith favorable thermal mass or light-dispersing properties, and organizing spaces that naturally circulate air.', metadata={'source': 'type_solar.pdf', 'file_path': 'type_solar.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20250314025842Z', 'modDate': '', 'trapped': ''}),
 Document(page_content="Solar energy is the radiant energy from the Sun's light and heat, which can be harnessed using a range of technologies such as solar electricity, \nsolar thermal energy (including solar water heating) and solar architecture. It is an essential source of re

**The above retrival without similarity index based returns the top one as the relevant document**

## Task 1.3 Explore the use of other text-generation models

In [7]:
!pip uninstall -y langchain langchain-core langchain-groq

Found existing installation: langchain 0.1.2
Uninstalling langchain-0.1.2:
  Successfully uninstalled langchain-0.1.2
Found existing installation: langchain-core 0.3.45
Uninstalling langchain-core-0.3.45:
  Successfully uninstalled langchain-core-0.3.45


In [3]:
# !pip install langchain==0.1.2

In [13]:
# !pip install llama-index
# !pip install llama-index-llms-groq
# !pip install llama-index-embeddings-huggingface
# !pip install llama-parse

In [4]:
import getpass
import os
#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
os.environ["GROQ_API_KEY"] = "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"

In [5]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-8b-8192")
llm_70b = Groq(model="llama3-70b-8192")

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

2025-03-14 14:11:29.528952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741961489.551483 4152585 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741961489.558501 4152585 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 14:11:29.584060: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
!pip install --upgrade llama-index-vector-stores-chroma

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [19]:
from langchain.vectorstores import FAISS
import os
from langchain.vectorstores import Chroma
import chromadb

# Assuming 'documents' is your list of Document objects
texts = [doc.page_content for doc in documents]
metadatas = [doc.metadata for doc in documents]
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5",  model_kwargs={"trust_remote_code": True})
# Initialize Chroma client
client = 0
client = chromadb.Client()

#Set your collection name (make sure it's a string)
for i, doc in enumerate(documents):
    collection_name = f"collection_{i+20}"
    collection = client.create_collection(collection_name)
    collection.add(
        documents=doc.page_content,
        ids=[str(i)]
    )

In [14]:
from langchain.vectorstores import Chroma
import chromadb
from typing import List
from transformers import AutoTokenizer, AutoModel
import torch

class CustomChromaManager:
    def __init__(self, model_name: str, model_kwargs: dict = None, encode_kwargs: dict = None):
        self.model_name = model_name
        self.model_kwargs = model_kwargs or {}
        self.encode_kwargs = encode_kwargs or {}
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, **self.model_kwargs)
        self.model = AutoModel.from_pretrained(model_name, **self.model_kwargs)
        self.model.eval()  # Set the model to evaluation mode

        # Initialize Chroma client
        self.client = chromadb.Client()

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state.mean(dim=1)
        return embeddings.cpu().numpy().tolist()

    def embed_query(self, query: str) -> List[float]:
        return self.embed_documents([query])[0]

    def create_and_add_documents(self, documents):
        # Group documents by collection name
        collections = {}
        for i, doc in enumerate(documents):
            collection_name = f"collection_{i+20}"
            if collection_name not in collections:
                collections[collection_name] = {'texts': [], 'ids': []}
            collections[collection_name]['texts'].append(doc.page_content)
            collections[collection_name]['ids'].append(str(i))

        # Create collections and add documents
        for collection_name, data in collections.items():
            collection = self.client.create_collection(collection_name)
            embeddings = self.embed_documents(data['texts'])
            collection.add(
                documents=data['texts'],
                ids=data['ids'],
                embeddings=embeddings
            )

    def similarity_search(self, query: str, collection_name: str, k: int = 5):
        collection = self.client.get_collection(collection_name)
        query_embedding = self.embed_query(query)
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=k
        )
        return results['documents']


In [17]:
# Initialize the custom Chroma manager
chroma_manager = CustomChromaManager(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={"trust_remote_code": True},
    encode_kwargs={"normalize_embeddings": True}
)

# Assuming 'documents' is your list of Document objects
chroma_manager.create_and_add_documents(documents)

# Perform a similarity search on a specific collection
query = "What is solar energy?"
collection_name = "collection_20"  # Replace with your collection name
results = chroma_manager.similarity_search(query, collection_name)

# Display the results
for result in results:
    print(result)


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


["Solar energy is the radiant energy from the Sun's light and heat, which can be harnessed using a range of technologies such as solar electricity, \nsolar thermal energy (including solar water heating) and solar architecture. It is an essential source of renewable energy, and its technologies \nare broadly characterized as either passive solar or active solar depending on how they capture and distribute solar energy or convert it into \nsolar power."]


## Task 2.1 Provide a list of the retriever and generator models you have utilized.

In Our setup, you've integrated multiple retrievers and generators to enhance your application's capabilities. Here's a breakdown of each component:

Retrievers:
* Instruction-Based Similarity Distance Retriever:
        This retriever utilizes a similarity metric, such as cosine similarity or Euclidean distance, to find documents that closely match the provided instruction or query. It's particularly effective when you have a well-defined set of documents and need to retrieve those most relevant to a specific instruction.
* Indexed Retriever:
        An indexed retriever relies on a pre-built index of documents, enabling efficient retrieval based on exact matches or keyword searches. This method is suitable when your dataset is relatively static, and you require fast retrieval based on specific terms or phrases.
* Groq Retriever:
        The Groq retriever leverages Groq's AI inference technology to perform rapid and efficient document retrieval. By utilizing Groq's Language Processing Unit (LPU), it accelerates the retrieval process, making it suitable for applications requiring high-performance inference.

Generators:
* Instruction-Based Generator:
        This generator is designed to produce responses or content based on specific instructions. By interpreting the given instruction, it generates relevant and contextually appropriate text, making it useful for tasks like content creation, summarization, or Q&A systems.
* Groq API Generator:
        The Groq API generator utilizes Groq's advanced AI processing capabilities to generate content. By interfacing with Groq's API, it can produce high-quality text responses, benefiting applications that require scalable and efficient text generation.

By combining these retrievers and generators, the application can effectively handle a wide range of tasks, from precise document retrieval to dynamic content generation, all while maintaining high performance and relevance to user queries.

## Task 2.2 Analyze any issues related to the models providing unrelated information

In [33]:
!pip install langchain==0.1.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain-0.1.2-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.83-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.51-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.50-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.49-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.48-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.47-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.46-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is still looking at multiple versions of langchain-cor

## Memory

In [52]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [53]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

**Conversation Buffer**

In [54]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [55]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

**Conversation Buffer Window**

In [56]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## Chain

In [5]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
# from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = 'lmsys/fastchat-t5-3b-v1.0'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id


2025-03-14 22:24:10.393285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741991050.417474    2835 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741991050.424966    2835 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 22:24:10.453141: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/jupyter-st125490/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_down

In [6]:
from huggingface_hub import hf_hub_download
from transformers import BitsAndBytesConfig
# Load tokenizer and model
model_id = "lmsys/fastchat-t5-3b-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set up BitsAndBytes for 4-bit quantization
# bitsandbyte_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=True
# )


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
tokenizer.pad_token_id = tokenizer.eos_token_id

# bitsandbyte_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type = "nf4",
#     load_in_8bit_fp32_cpu_offload=True,
#     bnb_4bit_compute_dtype = torch.float16,
#     bnb_4bit_use_double_quant = True
# )

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    device_map = 'auto'
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

In [7]:
#now let us load all the 10 questions to answer
# 1) How old are you?
# 2) What is your highest level of education?
# 3) What major or field of study did you pursue during your education?
# 4) How many years of work experience do you have?
# 5) What type of work or industry have you been involved in?
# 6) Can you describe your current role or job responsibilities?
# 7) What are your core beliefs regarding the role of technology in shaping society?
# 8) How do you think cultural values should influence technological advancements ?
# 9) As a master’s student, what is the most challenging aspect of your studies so far?
# 10) What specific research interests or academic goals do you hope to achieve during your time as a
# master’s student

In [8]:
texts = ["I supppose you are 29 year old", 
         "The hiher education i have is graduation",
         "The decipline of study is computer science",
         "approximatly 4 years of working expereince, do you want to know more about",
         "The working industry or area was IT and Finance majorly expereince as data anlyst and Data engineer at fintech startups",
         "Currently as if i am pursing masters so industrial role is out of the question",
         "Technology can shape the soceity with their daily needs, you see the paper pin to the big aeroplans all humans are these days relays on technology and to the matter of fact in order to create the better world around us, any other question you have", 
        "Culture and technology can go hand in hand, with that said culture is a traditional to the human value or create a ethical behaviour (to say what is wrong or right), where as technology is need of thd hour can have its own pros and cons",
         "As a masters student the most difficult part is coming back to the idea of acadamic thinking while if one have world in industry for a while, could be a little challanging", 
         "The majory research interest area i want to work is the non-subjective matters of solving AI problems, that is niche to the field rather than going with application that everybody is doing"
        ]

In [57]:
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define a template to answer questions about solar energy based on a single article
personal_prompt_template = PromptTemplate(
    input_variables=[],  # No dynamic input variables in this case
    template="""
    You are an assistant to provide information about myself, please ask questions:
    1) How old are you?
    2) What is your highest level of education?
    3) What major or field of study did you pursue during your education?
    4) How many years of work experience do you have?
    5) What type of work or industry have you been involved in?
    6) Can you describe your current role or job responsibilities?
    7) What are your core beliefs regarding the role of technology in shaping society?
    8) How do you think cultural values should influence technological advancements?
    9) As a master’s student, what is the most challenging aspect of your studies so far?
    10) What specific research interests or academic goals do you hope to achieve during your time as a master's student?

    Answer:
    """
)

# PROMPT = PromptTemplate.from_template(
#     template = personal_prompt_template
# )

documents = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=100
)
# Process each document and split into chunks
docs = text_splitter.create_documents(texts)
for document in docs:
    chunks = text_splitter.split_text(document.page_content)
    # print(chunks)
    # doc = text_splitter.split_documents(chunks)
    # Append all chunks to the list
    documents.extend(chunks)

doc = text_splitter.split_documents(docs)

for chunk in doc:
    print(chunk)
    print("-" * 20)


page_content='I supppose you are 29 year old'
--------------------
page_content='The hiher education i have is graduation'
--------------------
page_content='The decipline of study is computer science'
--------------------
page_content='approximatly 4 years of working expereince, do you want to know more about'
--------------------
page_content='The working industry or area was IT and Finance majorly expereince as data anlyst and Data engineer at fintech startups'
--------------------
page_content='Currently as if i am pursing masters so industrial role is out of the question'
--------------------
page_content='Technology can shape the soceity with their daily needs, you see the paper pin to the big aeroplans all humans are these days relays on technology and to the matter of fact in order to create the better world around us, any other question you have'
--------------------
page_content='Culture and technology can go hand in hand, with that said culture is a traditional to the human 

In [116]:
prompt.format(
    context = "The working industry or area was IT and Finance majorly expereince as data anlyst and Data engineer at fintech startups" ,
    question = "Which company i was involves with"
)

"\n    You are an assistant to provide information about myself, please ask questions:\n    1) How old are you?\n    2) What is your highest level of education?\n    3) What major or field of study did you pursue during your education?\n    4) How many years of work experience do you have?\n    5) What type of work or industry have you been involved in?\n    6) Can you describe your current role or job responsibilities?\n    7) What are your core beliefs regarding the role of technology in shaping society?\n    8) How do you think cultural values should influence technological advancements?\n    9) As a master’s student, what is the most challenging aspect of your studies so far?\n    10) What specific research interests or academic goals do you hope to achieve during your time as a master's student?\n\n    Context: The working industry or area was IT and Finance majorly expereince as data anlyst and Data engineer at fintech startups\n    "

In [23]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [13]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [46]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [47]:
query = 'Comparing both of them'
chat_history = "Human:What is my age?\nAI:\nHuman:Which industry i was involved with?\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is my age?
AI:
Human:Which industry i was involved with?
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is my age?\nAI:\nHuman:Which industry i was involved with?\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  your  age?\n'}

In [118]:
import os
texts = [d.page_content for d in doc]
embeddings = embedding_model.embed_documents(texts)

# Create a list of tuples (text, embedding)
text_embedding_pairs = zip(texts, embeddings)

# Initialize FAISS Vector Store from embeddings
vector_path = './vector_db_path_paersonal'  # Replace with your desired path
db_file_name = 'nlp_stanford'
vector_path = os.path.abspath(vector_path)
full_path = os.path.join(vector_path, db_file_name)
print(f"Full path where index is saved: {full_path}")

vectordb = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,
    embedding=embedding_model)

# Save the FAISS index to a specified local path
vectordb.save_local(
    folder_path=full_path,
    index_name='nlp'  # Default index name
)

print(f"FAISS index saved to: {os.path.join(vector_path, db_file_name)}")

Full path where index is saved: /home/jupyter-st125490/vector_db_path_paersonal/nlp_stanford
FAISS index saved to: /home/jupyter-st125490/vector_db_path_paersonal/nlp_stanford


In [120]:
vectordb = FAISS.load_local(
    folder_path = full_path,
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   
#ready to use
retriever = vectordb.as_retriever()

In [134]:

# Define memory for conversation history (stores last 3 interactions)
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # Key for the conversation history
    output_key="response",      # Key for the model's output
    k=3,  # Keep last 3 interactions
    return_messages=True
)

# Define the prompt template with input variables
prompt_temp = """
    You are an assistant to provide information about myself, please ask questions:
    1) How old are you?
    2) What is your highest level of education?
    3) What major or field of study did you pursue during your education?
    4) How many years of work experience do you have?
    5) What type of work or industry have you been involved in?
    6) Can you describe your current role or job responsibilities?
    7) What are your core beliefs regarding the role of technology in shaping society?
    8) How do you think cultural values should influence technological advancements?
    9) As a master’s student, what is the most challenging aspect of your studies so far?
    10) What specific research interests or academic goals do you hope to achieve during your time as a master's student?

    {context}
    Question: {question}
    Answer:
    """.strip()

prompt = PromptTemplate.from_template(
    template = prompt_temp)


# Create LLMChain with memory and the prompt
llm_chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

# Example query with context
query = "What type of work or industry have you been involved in?"
context = "The working industry or area was IT and Finance majorly expereince as data anlyst and Data engineer at fintech startups"



In [135]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = prompt,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant to provide information about myself, please ask questions:\n    1) How old are you?\n    2) What is your highest level of education?\n    3) What major or field of study did you pursue during your education?\n    4) How many years of work experience do you have?\n    5) What type of work or industry have you been involved in?\n    6) Can you describe your current role or job responsibilities?\n    7) What are your core beliefs regarding the role of technology in shaping society?\n    8) How do you think cultural values should influence technological advancements?\n    9) As a master’s student, what is the most challenging aspect of your studies so far?\n    10) What specific research interests or academic goals do you hope to achieve during your time as a master's student?\n\n    {context}\n    Question: {question}\n    Answer:

In [136]:
query = "How old are you?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant to provide information about myself, please ask questions:
    1) How old are you?
    2) What is your highest level of education?
    3) What major or field of study did you pursue during your education?
    4) How many years of work experience do you have?
    5) What type of work or industry have you been involved in?
    6) Can you describe your current role or job responsibilities?
    7) What are your core beliefs regarding the role of technology in shaping society?
    8) How do you think cultural values should influence technological advancements?
    9) As a master’s student, what is the most challenging aspect of your studies so far?
    10) What specific research interests or academic goals do you hope to achieve during your time as a master's student?

    I supppose you are 29 year old

approximatly 4 years of working expereince, do you want to kno

/home/jupyter-st125490/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.


{'input_documents': [Document(page_content='I supppose you are 29 year old'),
  Document(page_content='approximatly 4 years of working expereince, do you want to know more about'),
  Document(page_content='The hiher education i have is graduation'),
  Document(page_content='Currently as if i am pursing masters so industrial role is out of the question')],
 'question': 'How old are you?',
 'output_text': '<pad> Answer:  29\n'}

In [137]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant to provide information about myself, please ask questions:\n    1) How old are you?\n    2) What is your highest level of education?\n    3) What major or field of study did you pursue during your education?\n    4) How many years of work experience do you have?\n    5) What type of work or industry have you been involved in?\n    6) Can you describe your current role or job responsibilities?\n    7) What are your core beliefs regarding the role of technology in shaping society?\n    8) How do you think cultural values should influence technological advancements?\n    9) As a master’s student, what is the most challenging aspect of your studies so far?\n 

## chatbot

In [139]:
query = "what is my age"
answer = chain({"question":query})
answer




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant to provide information about myself, please ask questions:
    1) How old are you?
    2) What is your highest level of education?
    3) What major or field of study did you pursue during your education?
    4) How many years of work experience do you have?
    5) What type of work or industry have you been involved in?
    6) Can you describe your current role or job responsibilities?
    7) What are your core beliefs regarding the role of technology in shaping society?
    8) How do you think cultural values should influence technological advancements?
    9) As a master’s student, what is the most challenging aspect of your studies so far?
    10) What specific research interests or academic goals do you hope to achieve during your time as a master's student?

    I supppose you are 29 year old

approxi

/home/jupyter-st125490/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'what is my age',
 'chat_history': [],
 'answer': '<pad> Your  age  is  29.\n',
 'source_documents': [Document(page_content='I supppose you are 29 year old'),
  Document(page_content='approximatly 4 years of working expereince, do you want to know more about'),
  Document(page_content='The hiher education i have is graduation'),
  Document(page_content='The decipline of study is computer science')]}

In [140]:
query = "which comany i was involved with?"
answer = chain({"question":query})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='what is my age'), AIMessage(content='<pad> Your  age  is  29.\n')]
Follow Up Input: which comany i was involved with?
Standalone question:


/home/jupyter-st125490/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant to provide information about myself, please ask questions:
    1) How old are you?
    2) What is your highest level of education?
    3) What major or field of study did you pursue during your education?
    4) How many years of work experience do you have?
    5) What type of work or industry have you been involved in?
    6) Can you describe your current role or job responsibilities?
    7) What are your core beliefs regarding the role of technology in shaping society?
    8) How do you think cultural values should influence technological advancements?
    9) As a master’s student, what is the most challenging aspect of your studies so far?
    10) What specific research interests or academic goals do you hope to achieve during your time as a master's student?

    The working industry or area was IT and Finance majorly expereince as data 

/home/jupyter-st125490/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'which comany i was involved with?',
 'chat_history': [HumanMessage(content='what is my age'),
  AIMessage(content='<pad> Your  age  is  29.\n')],
 'answer': '<pad> The  companies  I  was  involved  with  were  fintech  startups.\n',
 'source_documents': [Document(page_content='The working industry or area was IT and Finance majorly expereince as data anlyst and Data engineer at fintech startups'),
  Document(page_content='approximatly 4 years of working expereince, do you want to know more about'),
  Document(page_content='Currently as if i am pursing masters so industrial role is out of the question'),
  Document(page_content='The majory research interest area i want to work is the non-subjective matters of solving AI problems, that is niche to the field rather than going with application that everybody is doing')]}

In [141]:
query = " What specific research interests?"
answer = chain({"question":query})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='what is my age'), AIMessage(content='<pad> Your  age  is  29.\n'), HumanMessage(content='which comany i was involved with?'), AIMessage(content='<pad> The  companies  I  was  involved  with  were  fintech  startups.\n')]
Follow Up Input:  What specific research interests?
Standalone question:


/home/jupyter-st125490/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant to provide information about myself, please ask questions:
    1) How old are you?
    2) What is your highest level of education?
    3) What major or field of study did you pursue during your education?
    4) How many years of work experience do you have?
    5) What type of work or industry have you been involved in?
    6) Can you describe your current role or job responsibilities?
    7) What are your core beliefs regarding the role of technology in shaping society?
    8) How do you think cultural values should influence technological advancements?
    9) As a master’s student, what is the most challenging aspect of your studies so far?
    10) What specific research interests or academic goals do you hope to achieve during your time as a master's student?

    The majory research interest area i want to work is the non-subjective matt

/home/jupyter-st125490/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': ' What specific research interests?',
 'chat_history': [HumanMessage(content='what is my age'),
  AIMessage(content='<pad> Your  age  is  29.\n'),
  HumanMessage(content='which comany i was involved with?'),
  AIMessage(content='<pad> The  companies  I  was  involved  with  were  fintech  startups.\n')],
 'answer': "<pad>   pad>  My  current  research  interests  are  in  the  non-subjective  aspects  of  solving  AI  problems,  specifically  in  the  area  of  natural  language  processing  and  machine  learning.  I  am  particularly  interested  in  exploring  the  intersection  of  these  two  fields  and  how  they  can  be  used  to  solve  real-world  problems.  I  am  also  interested  in  exploring  the  potential  of  AI  to  improve  healthcare  and  other  areas  of  human  life.  I  am  also  interested  in  exploring  the  potential  of  AI  to  improve  the  quality  of  life  for  people  around  the  world.  I  am  currently  pursuing  a  master's  degree 

## Task 2.2 Analyze any issues related to the models providing unrelated information?

If you're encountering issues where the models are providing **unrelated information**, here are a few areas to consider and strategies to help mitigate those issues:

### 1. **Issues with Document Retrieval**:
   - **Problem**: The retriever might not be pulling the most relevant documents or content, leading to irrelevant responses.
   - Or the possibility of having no releavent documents avilable,
   - **Solution**: Removing the irrelavent context and stay in-context will return right answers

### 2. **Inadequate Prompt Engineering**:
   - **Problem**: The way the model is prompted could be insufficient to guide it to return the correct information.
   - **Solution**: Here we need to define the propoer prompt structure which helps the model to return the relevant information
     
### 3. **Model Over-Generation or Hallucination**:
   - **Problem**: The model might be generating overly verbose, non-relevant information, commonly referred to as "hallucinations."
   - **Solutions**:
     - **Limit the response length**: Explicitly tell the model to give concise answers. For example, you could instruct it to "Answer in no more than 2 sentences."
     - **Fine-tune on domain-specific data**: Fine-tuning your model on domain-specific content (e.g., information related to your query type) can help reduce hallucinations and improve accuracy.
     - **Apply filtering**: Use an additional filtering step to check if the model's generated output aligns with the expected output (e.g., filter out non-related answers).

### 4. **Model's Lack of Domain Knowledge**:
   - **Problem**: The model may not have the necessary knowledge about the context you're asking it to analyze, especially if it doesn't have enough training data in the relevant domain.
   - **Solutions**:
     - **Domain-specific fine-tuning**: Fine-tune the model on data that closely matches your use case (e.g., using custom corpora for a specific field like AI or finance).
     - **Utilize domain-specific models**: Use models that have been trained on domain-specific data to increase accuracy (e.g., finance-oriented models for financial questions).

### 5. **Ambiguity in the Query**:
   - **Problem**: The question might be too vague or ambiguous, causing the model to provide a less targeted answer.
   - **Solutions**:
     - **Clarify the question**: Refine and specify the question better. For example, "What is my age based on the context I provided?" instead of just "How old are you?"
     - **Use disambiguation**: Ask for more specific outputs by adding more context or conditions in the question to guide the model toward a precise answer.

### 6. **Inconsistent Memory or State Management**:
   - **Problem**: If your model or retriever is using memory but is not properly managing the state of the conversation or context, this could lead to irrelevant outputs.
   - **Solutions**:
     - **Check memory and context management**: Ensure that the memory management of the model is correctly implemented. For example, verify that the relevant parts of the context are updated and available during each interaction.
     - **Limit context size**: If memory becomes too large or unstructured, the model may lose focus on the critical information. Limit the memory to the most relevant and recent exchanges.

### 7. **Model Hyperparameters**:
   - **Problem**: The model's parameters or temperature settings might not be properly configured for the task, causing it to produce irrelevant or verbose outputs.
   - **Solutions**:
     - **Adjust temperature and top-k settings**: Reduce the temperature for more deterministic and focused responses (e.g., temperature 0.3–0.5).
     - **Set appropriate token limits**: Adjust the `max tokens` parameter to ensure that the model’s response remains concise and on-topic.

---


## The insights to the work

 * Here we work two datasets one is sourced from wikipedia about solar energy another dataset is our personal information which we have to build prompt for
 * We use two generators and two retrivers , one is from lama model , groq model using groq api we did proceed with that since we have package in compatability , installed the packages independently
 * The another one is the instruct-base which comes from tutorial notebook
 * The limitation to the task is to extract the irrelevat infomration or thresholding since we tried that but was not able to get the score
 * So this solution will give same results to the questions like "What is solar energy?, what is solar panal?, what is solar system? with the instruct-base embeeding"
 * Futher imporvements can be done using chain-of-thought or few shot learning for better reasoning